# Step 0 : Downloading/loading libraries and data 

## Downloading data

In [33]:
# Downloading tweets 
! wget http://172.22.218.81/tweets+labels.tsv.gz

--2016-10-31 21:46:01--  http://172.22.218.81/tweets+labels.tsv.gz
Connecting to 172.22.218.81:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79966529 (76M) [application/octet-stream]
Saving to: ‘tweets+labels.tsv.gz’

100%[======================================>] 79,966,529  2.11MB/s   in 31s    

2016-10-31 21:46:32 (2.48 MB/s) - ‘tweets+labels.tsv.gz’ saved [79966529/79966529]



In [34]:
## Extracting and cleaning tweets 
! gunzip tweets+labels.tsv.gz 
! cat tweets+labels.tsv | tr -d "\r" > cleaned_dataset.tsv

## Downloading libraries

In [35]:
# Downloading required nltk data 
nltk.download('punkt') ## data used for the word tokenizer 
nltk.download("wordnet") ## data used for the lemmatizer

[nltk_data] Downloading package punkt to /home/ds/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ds/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Loading libraries

In [36]:
# pandas for data manipulation 
import pandas as pd
# nltk for natural language utilities 
from nltk import word_tokenize, WordNetLemmatizer

# Step 1 : loading data 

In [37]:
data = pd.read_csv("cleaned_dataset.tsv", sep= "\t", header=None, names=["id", "sentiment", "text"])

In [38]:
## Sampling 6800 tweets positive / negative / neutral 
pos = data[data["sentiment"] == "positive"].sample(6800)
neg = data[data["sentiment"] == "negative"].sample(6800)
neu = data[data["sentiment"] == "neutral"].sample(6800)

In [39]:
## Combining pos, neg, neu 
dat = pd.concat([pos, neg, neu]).sample(frac=1)

# Step 2 : Preprocessing

### linking the different forms of the same word (for example, price and prices, is and are) to each other – the tools that can do that are called lemmatizers, and you can again use one of those that come with the NLTK.

### converting all words to lowercase so that the classifier does not treat People, people and PEOPLE as three separate features.

### Splitting the text by white spaces and punctuation marks – the tools that are used for this purpose are called tokenizers, and you can use a tokenizer provided with the NLTK.

### Removing stop words linked to English language

### Removing URLs

### Removing special characters 

### Removing punctuation 

# Step 3 : Feature extraction

In [23]:
wordnet_lemmatizer = WordNetLemmatizer()

In [24]:
def preprocess(sentence):
    return [wordnet_lemmatizer.lemmatize(word.lower()) for word in word_tokenize(sentence.decode('utf-8'))]

In [173]:
tokenizer = lambda x: nltk.word_tokenize(x.decode('utf-8'))

In [29]:
dat["text"] = dat["text"].apply(preprocess)

In [186]:
# Feature extraction

In [187]:
from nltk.corpus import stopwords

In [189]:
stoplist = stopwords.words("english")

In [191]:
twitter_stop_words = ["@", "rt"]

In [196]:
remove_stop_word = lambda list_of_words: [word for word in list_of_words if word not in stoplist+twitter_stop_words]

In [197]:
remove_stop_word(["a", "dog"])

['dog']

In [198]:
dat["text"] = dat["text"].apply(remove_stop_word)

In [201]:
from collections import Counter

In [202]:
def get_features(text, setting):
    if setting=='bow':
        return {word: count for word, count in Counter(preprocess(text)).items() if not word in stoplist}
    else:
        return {word: True for word in preprocess(text) if not word in stoplist}

In [ ]:
all_features = [(get_features(email, 'bow'), label) for (email, label) in all_emails]